<a href="https://colab.research.google.com/github/siva5271/week_16/blob/master/sentiment_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split,GridSearchCV,cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report
import pandas as pd
import numpy as np
import nltk
import spacy
nlp=spacy.load("en_core_web_sm")

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
df=pd.read_csv('/content/drive/MyDrive/datasets/kindle_reviews.csv')
df

,Unnamed: 0,asin,helpful,overall,reviewText,reviewTime,reviewerID,reviewerName,summary,unixReviewTime
0,0,B000F83SZQ,"[0, 0]",5,I enjoy vintage books and movies so I enjoyed ...,"05 5, 2014",A1F6404F1VG29J,Avidreader,Nice vintage story,1399248000
1,1,B000F83SZQ,"[2, 2]",4,This book is a reissue of an old one; the auth...,"01 6, 2014",AN0N05A9LIJEQ,critters,Different...,1388966400
2,2,B000F83SZQ,"[2, 2]",4,This was a fairly interesting read. It had ol...,"04 4, 2014",A795DMNCJILA6,dot,Oldie,1396569600
3,3,B000F83SZQ,"[1, 1]",5,I'd never read any of the Amy Brewster mysteri...,"02 19, 2014",A1FV0SX13TWVXQ,"Elaine H. Turley ""Montana Songbird""",I really liked it.,1392768000
4,4,B000F83SZQ,"[0, 1]",4,"If you like period pieces - clothing, lingo, y...","03 19, 2014",A3SPTOKDG7WBLN,Father Dowling Fan,Period Mystery,1395187200
...,...,...,...,...,...,...,...,...,...,...
982614,982614,B00M13FNSS,"[2, 2]",5,Yasss hunny! This is a great read. That Dre is...,"07 23, 2014",A2Y66HD4J5S7QZ,Candi,A Hot Read Indeed!!,1406073600
982615,982615,B00M13FNSS,"[0, 0]",5,I ENJOYED THIS BOOK FROM BEGINNING TO END NOW ...,"07 23, 2014",A17YHECC8H9NEY,Margie,VERY GOOD BOOK,1406073600
982616,982616,B00M13FNSS,"[1, 1]",5,Great book! Cherika was a fool. She let that m...,"07 23, 2014",A20KO0BPMNREJL,Nicki,Great Read,1406073600
982617,982617,B00M13FNSS,"[0, 0]",5,When I say this was an excellent book please b...,"07 23, 2014",A1BQO66R6OLCCW,Nikey,Wow!!,1406073600


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 982619 entries, 0 to 982618
Data columns (total 10 columns):
 #   Column          Non-Null Count   Dtype 
---  ------          --------------   ----- 
 0   Unnamed: 0      982619 non-null  int64 
 1   asin            982619 non-null  object
 2   helpful         982619 non-null  object
 3   overall         982619 non-null  int64 
 4   reviewText      982597 non-null  object
 5   reviewTime      982619 non-null  object
 6   reviewerID      982619 non-null  object
 7   reviewerName    978803 non-null  object
 8   summary         982618 non-null  object
 9   unixReviewTime  982619 non-null  int64 
dtypes: int64(3), object(7)
memory usage: 75.0+ MB


In [ ]:
df.dropna(subset=['reviewText'],axis=0,inplace=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 982597 entries, 0 to 982618
Data columns (total 10 columns):
 #   Column          Non-Null Count   Dtype 
---  ------          --------------   ----- 
 0   Unnamed: 0      982597 non-null  int64 
 1   asin            982597 non-null  object
 2   helpful         982597 non-null  object
 3   overall         982597 non-null  int64 
 4   reviewText      982597 non-null  object
 5   reviewTime      982597 non-null  object
 6   reviewerID      982597 non-null  object
 7   reviewerName    978781 non-null  object
 8   summary         982596 non-null  object
 9   unixReviewTime  982597 non-null  int64 
dtypes: int64(3), object(7)
memory usage: 82.5+ MB


In [ ]:
df['overall'].value_counts()

5    575246
4    254010
3     96193
2     34130
1     23018
Name: overall, dtype: int64

In [ ]:
df['overall'].unique()

array([5, 4, 3, 2, 1])

In [ ]:
df_sampled=pd.DataFrame(columns=df.columns)
for category in df['overall'].unique():
    df_temp=df[df['overall']==category].sample(23018)
    df_sampled=pd.concat([df_sampled,df_temp],axis=0)
df_sampled

,Unnamed: 0,asin,helpful,overall,reviewText,reviewTime,reviewerID,reviewerName,summary,unixReviewTime
914744,914744,B00K12UUES,"[0, 0]",5,Three sisters who has more drama than a little...,"06 4, 2014",A3OI13SD7AB6VH,MZ.URAQT,DRAMAAAAAAAAAAAAAAAAAAAAAAAA!!!!,1401840000
505148,505148,B00CIIZPAW,"[1, 1]",5,In Another Life - Jo A CattellI was given an A...,"06 16, 2013",A3NHC2IWEU4Z6S,Peter Martin,Emotion-filled 5*****,1371340800
279307,279307,B008W0IXZ4,"[0, 0]",5,Guess what? Mr. Stevens well get my vote anyti...,"03 3, 2014",ADPBIOM9X3IN5,booboo,Booboo,1393804800
380125,380125,B00AO00VBE,"[1, 1]",5,Although this is described as a young adult se...,"07 18, 2013",A3QGHV1LS3NL5D,"Pat Z ""ngmabooks""",What a great series!!,1374105600
653598,653598,B00F9AO81S,"[1, 1]",5,This was such a different take on the story to...,"03 18, 2014",A1IIOY9IA47UB1,Bette Hillner,What a delight!,1395100800
...,...,...,...,...,...,...,...,...,...,...
656825,656825,B00FB1616A,"[1, 1]",1,Where do i start...i seriously wanted to like ...,"06 2, 2014",A1WJ7CZWGGPW8L,Mher,DNF,1401667200
784931,784931,B00HRIK2BU,"[1, 1]",1,the story was to short the plot was not fully ...,"02 16, 2014",A1DE9LILYOGSRB,barbara,to short,1392508800
255447,255447,B008H459W0,"[0, 0]",1,I expected a decent book only to be given less...,"12 7, 2013",ANU2ZQ5ZPBQL1,Janice,Terrible,1386374400
135361,135361,B0067O8L6S,"[0, 0]",1,"The poems and stories might be all right, if a...","01 3, 2012",A3INL3IB0KS081,myjunk,emotional poems,1325548800


In [ ]:
df_sampled.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 115090 entries, 914744 to 150749
Data columns (total 10 columns):
 #   Column          Non-Null Count   Dtype 
---  ------          --------------   ----- 
 0   Unnamed: 0      115090 non-null  object
 1   asin            115090 non-null  object
 2   helpful         115090 non-null  object
 3   overall         115090 non-null  object
 4   reviewText      115090 non-null  object
 5   reviewTime      115090 non-null  object
 6   reviewerID      115090 non-null  object
 7   reviewerName    114493 non-null  object
 8   summary         115090 non-null  object
 9   unixReviewTime  115090 non-null  object
dtypes: object(10)
memory usage: 9.7+ MB


In [ ]:
def preprocess(txt):
  doc=nlp(txt)
  filtered_tokens=[]
  for tokens in  doc:
    if tokens.is_stop or tokens.is_punct:
      continue
    filtered_tokens.append(tokens.lemma_)
  return ' '.join(filtered_tokens)

In [ ]:
# df_sampled['reviewText']=df_sampled['reviewText'].apply(lambda x:preprocess(x))
df_sampled['overall']=df_sampled['overall'].apply(lambda x:int(x))

In [ ]:
# clf=Pipeline([
#     ('tfidf',TfidfVectorizer(ngram_range=(1,3))),
#     ('naive_bayes',MultinomialNB())
# ])
# X_train,X_test,y_train,y_test=train_test_split(df_sampled['reviewText'],\
#                                                df_sampled['overall'],test_size=0.2,stratify=df_sampled.overall)
# clf.fit(X_train,y_train)
# y_pred=clf.predict(X_test)
# print(classification_report(y_test,y_pred))

In [ ]:
# clf=Pipeline([
#     ('tfidf',TfidfVectorizer()),
#     ('naive_bayes',RandomForestClassifier())
# ])
# clf.fit(X_train,y_train)
# y_pred=clf.predict(X_test)
# print(classification_report(y_test,y_pred))

In [ ]:
# models={
#     'random_forest':{
#         'model':RandomForestClassifier(),
#         'params':{
#             'classifier__n_estimators':[50,100,150,200,250],
#             'tfidf__ngram_range':[(1,1),(1,2),(1,3)]
#         }
#     },
#     'support_vector_machine':{
#         'model':SVC(),
#         'params':{
#             'tfidf__ngram_range':[(1,1),(1,2),(1,3)]
#         }
#     },
#     'k_nearest_neighbors':{
#         'model':KNeighborsClassifier(),
#         'params':{
#            'classifier__n_neighbors':[1,3,5,10,20,50],
#            'tfidf__ngram_range':[(1,1),(1,2),(1,3)]
#         }
#     },
#     'naive_bayes':{
#         'model':MultinomialNB(),
#         'params':{
#             'tfidf__ngram_range':[(1,1),(1,2),(1,3)]
#         }
#     }
# }

In [ ]:
# scores={}
# for k,v in models.items():
#   clf=Pipeline([
#       ('tfidf',TfidfVectorizer()),
#       ('classifier',v['model'])
#   ])
#   grid=GridSearchCV(clf,v['params'],cv=5)
#   grid.fit(df_sampled['reviewText'],df_sampled['overall'])
#   scores[k]={
#       'best_score':grid.best_score_,
#       'best_params':grid.best_params_
#   }
# scores

In [ ]:
clf=Pipeline([
    ('tfidf',TfidfVectorizer()),
    ('classifier',MultinomialNB())
])
score=cross_val_score(clf,df_sampled['reviewText'],df_sampled['overall'],cv=5)
score.mean()

0.5072030584759754

In [ ]:
  X_train,X_test,y_train,y_test=train_test_split(df_sampled['reviewText'],df_sampled['overall'],\
                                               test_size=0.2)
clf=Pipeline([
    ('tfidf',TfidfVectorizer()),
    ('classifier',SVC())
])
clf.fit(X_train,y_train)